# <span style="color:red">**exercise 3**</span>
![Python logo](./python_logo.gif)
---

### submited by:
- Name: Shahar Asher
- Id: 209305408
- Email adress: shaharas@edu.hac.ac.il
- Date: 19/04/2024

### Operation system: Windows 11
### Python version: 3.11.5 (Using anaconda)
### IDE: Visual Studio Code
### libraries: numpy, pandas, dataclasses, re, inspect
---

In [249]:
# imports
import pandas as pd
from dataclasses import dataclass, make_dataclass
import re
import numpy as np
import inspect

# Q.1

This script reads a Pandas DataFrame and generates a list of dataclass instances. Each dataclass instance has attributes corresponding to the DataFrame column names. If a column name is not a valid Python identifier, the script transforms the column name into a valid identifier.

**To accomplish this**:
- The script defines a function `make_valid_identifier` to generate valid Python identifiers from string values.
- Another function `create_instances` takes a Pandas DataFrame as input and dynamically creates a dataclass with field names derived from the DataFrame column names.
- It then iterates through the DataFrame rows and creates instances of the dynamically created dataclass.

In [250]:
def make_valid_identifier(value:str)->str:
    """
    Generates a valid Python identifier from a given string.

    Args:
        value (str): The string to convert into a valid identifier.

    Returns:
        str: The valid Python identifier derived from the input string.

    Example:
        >>> make_valid_identifier("Payment (US₪)")
        'nV_Payment_US'
    """
    identifier:str = 'nV_'
    identifier += re.sub(r'\W|^(?=\d)', '_', value)
    while(identifier.endswith('_')):
        identifier = identifier[:-1]
    
    for i in range(0, len(identifier)):
        if i+1 < len(identifier):
            if identifier[i] == '_' and identifier[i+1] == '_':
                identifier = identifier[:i] + identifier[i+1:]

    return identifier

In [251]:
def create_instances(df:pd.DataFrame)->dict:
    """
    Creates dataclass instances from a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The Pandas DataFrame containing the data.

    Returns:
        dict: A dictionary containing dataclass instances.

    Example:
        >>> 1 name sur name  Payment (US₪)
        ... 0   1    a        Q          12.30
        ... 1   4    b        B           4.20
        ... 2   6    c        !          -9.12
        ... 3   3    d        9           0.00
        ... 4  10    e        A           0.14
        >>> aa = DynamicClass(nV_1=1, nV_name='a', nV_sur_name='Q', nV_Payment_US=12.3)
        ... bb = DynamicClass(nV_1=4, nV_name='b', nV_sur_name='B', nV_Payment_US=4.2)
        ... cc = DynamicClass(nV_1=6, nV_name='c', nV_sur_name='!', nV_Payment_US=-9.12)
        ... dd = DynamicClass(nV_1=3, nV_name='d', nV_sur_name='9', nV_Payment_US=0.0)
        ... ee = DynamicClass(nV_1=10, nV_name='e', nV_sur_name='A', nV_Payment_US=0.14)
    """
    fields = [(make_valid_identifier(str(col)), type(df[col].iloc[0])) for col in df.columns]
    DataClass:type = make_dataclass("DynamicClass", fields)
    
    instances:dict = {}
    for i, row in enumerate(df.itertuples(index=False)):
        instance_name:str = chr(ord('a') + i)*2
        instances[instance_name] = DataClass(*row)
    
    return instances

In [252]:
data:dict = {1: [1,4,6,3,10],
        'name': ['a', 'b','c','d', 'e'],
        'sur name': ['Q','B','!','9','A'],
        'Payment (US₪)': [12.3, 4.2, -9.12, 0.0, 0.14]
        }

df:pd.DataFrame = pd.DataFrame(data)
instances:dict = create_instances(df)

# Print the instances
for name, instance in instances.items():
        print(f"{name} = {instance}")

aa = DynamicClass(nV_1=1, nV_name='a', nV_sur_name='Q', nV_Payment_US=12.3)
bb = DynamicClass(nV_1=4, nV_name='b', nV_sur_name='B', nV_Payment_US=4.2)
cc = DynamicClass(nV_1=6, nV_name='c', nV_sur_name='!', nV_Payment_US=-9.12)
dd = DynamicClass(nV_1=3, nV_name='d', nV_sur_name='9', nV_Payment_US=0.0)
ee = DynamicClass(nV_1=10, nV_name='e', nV_sur_name='A', nV_Payment_US=0.14)


---
# Q.2

This class is restricts the number of instances that can be created to a specified positive integer.

__To achieve this__:
- The script defines a class `LimitedInstances` that keeps track of the number of instances created.
- It ensures that no more than the specified maximum number of instances can be created.

In [253]:
class LimitedInstances:
    """
    Class that restricts the number of instances that can be created.

    Attributes:
        _instances (list): List to store created instances.
        _number_of_instances (list): List to store the maximum number of instances allowed.

    Methods:
        __init__: Initializes the class instance.
        __set_max_instances: Sets the maximum number of instances allowed.
        __get_max_instances: Retrieves the maximum number of instances allowed.
        __get_curent_number_of_instances: Retrieves the current number of instances created.
        __str__: Returns a string representation of the class instance.
    """
    _instances:list = []
    _number_of_instances:list = []
    
    def __init__(self, *args: int|None)->None:
        """
        Initializes the LimitedInstances class instance.

        Args:
            *args (int|None): Optional arguments to set the maximum number of instances allowed.

        Returns:
            None

        Raises:
            Exception: If the maximum number of instances is less than the current number of instances.
        """
        if len(args):
            self.__set_max_instances(args[0])
        if len(self._instances) < self._number_of_instances[0]:
            self._instances.append(self)
        else:
            raise Exception(f"Error: Cannot create more than {self._number_of_instances[0]} instances of: class {self.__class__.__name__}(self, *args)")
    

    def __set_max_instances(self, max_instances:int)->None:
        """
        Sets the maximum number of instances allowed.

        Args:
            max_instances (int): The maximum number of instances allowed.

        Returns:
            None

        Raises:
            Exception: If the maximum number of instances is less than the current number of instances.
        """
        if len(self._number_of_instances) and max_instances < len(self._instances):
            raise Exception(f"Error: Cannot set the maximum number of instances to less than the current number of instances: {self.__get_curent_number_of_instances()}")
        
        if len(self._number_of_instances):
            self._number_of_instances[0] = max_instances
        else:
            self._number_of_instances.append(max_instances)


    def __get_max_instances(self)->int:
        """
        Retrieves the maximum number of instances allowed.

        Returns:
            int: The maximum number of instances allowed.
        """
        return self._number_of_instances[0]
    

    def __get_curent_number_of_instances(self)->int:
        """
        Retrieves the current number of instances created.

        Returns:
            int: The current number of instances created.
        """
        return len(self._instances)


    def __str__(self)->str:
        """
        Returns a string representation of the class instance.

        Returns:
            str: A string representation of the class instance.

        Example:
            >>> instance = LimitedInstances(3)
            >>> print(instance)
            LimitedInstances has 0 instances, and can have at most 3 instances
        """
        return f"{self.__class__.__name__} has {len(self._instances)} instances, and can have at most {self.__get_max_instances()} instances"

In [254]:
try:
    def generate_and_max_instances(max_instances:int)->LimitedInstances:
        """
        Generates a LimitedInstances class instance with a specified maximum number of instances.

        Args:
            max_instances (int): The maximum number of instances allowed for the LimitedInstances class.

        Returns:
            LimitedInstances: An instance of the LimitedInstances class with the specified maximum number of instances.

        Raises:
            Exception: If there is an error creating the LimitedInstances instance.
        """
        temp_class:LimitedInstances = LimitedInstances(max_instances)
        return temp_class
except Exception as e:
    print(e)

In [255]:
try:
    temp_1:LimitedInstances = generate_and_max_instances(3)
    print(temp_1)
    temp_2:LimitedInstances = LimitedInstances()
    print(temp_2)
    temp_3:LimitedInstances = LimitedInstances()
    print(temp_3)
    temp_4:LimitedInstances = generate_and_max_instances(2)
    print(temp_4)
    temp_5:LimitedInstances = LimitedInstances()
    print(temp_5)

except Exception as e:
    print(e)

LimitedInstances has 1 instances, and can have at most 3 instances
LimitedInstances has 2 instances, and can have at most 3 instances
LimitedInstances has 3 instances, and can have at most 3 instances
Error: Cannot set the maximum number of instances to less than the current number of instances: 3


---
# Q.3

This factory function generates a class representing a cyclic field if the input integer is a prime number. The class implements arithmetic operations such as addition, subtraction, multiplication, and division.

To achieve this:
- The script defines a function `generate_cyclic` that checks if the input number is prime.
- If the number is prime, it dynamically creates a class `Cyclic` with arithmetic operations based on the input number.

In [256]:
class Cyclic:
    """
    Class representing a cyclic field with arithmetic operations.

    Methods:
        __init__: Initializes the Cyclic class instance.
        __is_prime: Checks if a number is prime.
        get_num: Retrieves the current value of the cyclic field.
        __add__: Adds a value to the cyclic field.
        __sub__: Subtracts a value from the cyclic field.
        __mul__: Multiplies the cyclic field by a value.
        __truediv__: Divides the cyclic field by a value.
    """
    def __init__(self, num:int)->None:
        """
        Initializes the Cyclic instance with the given number.

        Args:
            num (int): The initial value for the cyclic field.

        Returns:
            None

        Raises:
            ValueError: If the given number is not a prime number.
        """
        self.__num:int = num
        if not self.__is_prime():
            raise ValueError(f"Error: {num} is not a prime number.")
            
    
    def __is_prime(self)->bool:
        """
        Checks if the current number is prime.

        Returns:
            bool: True if the current number is prime, False otherwise.
        """
        if self.__num < 2:
            return False
        for n in range(2, int(np.sqrt(self.__num))+1):
            if self.__num % n == 0:
                return False
        return True


    def get_num(self)->int:
        """
        Retrieves the current value of the cyclic field.

        Returns:
            int: The current value of the cyclic field.

        Example:
            >>> cyclic = Cyclic(5)
            >>> cyclic.get_num()
            5
        """
        return self.__num
    
    
    def __add__(self, other)->int:
        """
        Adds a value to the cyclic field.

        Args:
            other (int|Cyclic): The value to add to the cyclic field.

        Returns:
            int: The updated value of the cyclic field.
        """
        if isinstance(other, int):
            other_num:int = other
        else:
            other_num:int = other.get_num()
        self.__num += other_num
        return self.get_num()
    
    
    def __sub__(self, other)->int:
        """
        Subtracts a value from the cyclic field.

        Args:
            other (int|Cyclic): The value to subtract from the cyclic field.

        Returns:
            int: The updated value of the cyclic field.
        """
        if isinstance(other, int):
            other_num:int = other
        else:
            other_num:int = other.get_num()
        self.__num -= other_num
        return self.get_num()
    
    
    def __mul__(self, other)->int:
        """
        Multiplies the cyclic field by a value.

        Args:
            other (int|Cyclic): The value to multiply the cyclic field by.

        Returns:
            int: The updated value of the cyclic field.
        """
        if isinstance(other, int):
            other_num:int = other
        else:
            other_num:int = other.get_num()
        self.__num *= other_num
        return self.get_num()
    
    
    def __truediv__(self, other)->int:
        """
        Divides the cyclic field by a value.

        Args:
            other (int|Cyclic): The value to divide the cyclic field by.

        Returns:
            int: The updated value of the cyclic field.

        Raises:
            ZeroDivisionError: If the divisor is zero.
        """
        if isinstance(other, int):
            other_num:int = other
        else:
            other_num:int = other.get_num()
        if other_num == 0:
            raise ZeroDivisionError("Error: Cannot divide by zero")
        self.__num //= other_num
        return self.get_num()

In [257]:
def generate_cyclic(num:int)->Cyclic:
    """
    Factory function to generate a Cyclic instance.

    Args:
        num (int): The initial value of the cyclic field.

    Returns:
        Cyclic: A Cyclic instance with the specified initial value.

    Example:
        >>> generate_cyclic(5)
        <Cyclic object at 0x000001E9EC708D90>
    """
    return Cyclic(num)

In [258]:
try:
    gen_1:Cyclic = generate_cyclic(5)
    gen_2:Cyclic = generate_cyclic(3)

    print(gen_1 + gen_2)
    print(gen_1 - gen_2)
    print(gen_1 * gen_2)
    print(gen_1 / gen_2)
    print(gen_1 + 100)

    gen_3:Cyclic = generate_cyclic(4)
except ZeroDivisionError as e:
    print(e)
except ValueError as e:
    print(e)

8
5
15
5
105
Error: 4 is not a prime number.


---
# Q.4
- The code is in the file *`salat.py`*

This module contains a singleton logger class that logs messages from multiple modules and stores them in a dictionary.

To achieve this:
- The script defines a class `GlobalLogger` as a singleton with a private constructor and a static method to retrieve the instance.
- It maintains a dictionary to store log messages associated with module names.
- Modules can log messages, which are stored in the dictionary under the respective module name.

```python
# import libraries
import inspect
import re
```

```python
class GlobalLogger:
    """
    Singleton logger class for logging messages.

    Attributes:
        _instance (GlobalLogger): Singleton instance of the GlobalLogger class.
        __messages_dict (dict): Dictionary to store log messages.

    Methods:
        __new__: Creates a new instance of the GlobalLogger class if it doesn't exist.
        __init__: Initializes the GlobalLogger instance.
        __my_file_name: Retrieves the name of the calling module.
        get_file_name: Retrieves the name of the module associated with the logger.
        message: Logs a message associated with the calling module.
        __getitem__: Retrieves the log messages associated with a specific module.
        __get_log_level: Retrieves the log level of the logger.
        log: Logs a message with the specified log level.
    """
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        """
        Creates a new instance of the GlobalLogger class if it doesn't exist.

        Args:
            *args: Positional arguments.
            **kwargs: Keyword arguments.

        Returns:
            GlobalLogger: The singleton instance of the GlobalLogger class.
        """
        if not cls._instance:
            cls._instance = super(GlobalLogger, cls).__new__(cls, *args, **kwargs)
            cls._instance.__messages_dict = {}
        return cls._instance
        
        
    def __init__(self)->None:
        """
        Initializes the GlobalLogger instance.

        Returns:
            None
        """
        self.__file_name:str = self.__my_file_name()
        self.__log_level:str = "LOG"
        if self.get_file_name() not in self.__messages_dict:
            self.__messages_dict:dict[str, list[str]] = {self.get_file_name(): []}
    
    
    def __my_file_name(self)->str:
        """
        Retrieves the name of the calling module.

        Returns:
            str: The name of the calling module.
        """
        module_name:str = inspect.currentframe().f_globals['__file__']
        file_name:str = re.search(r"(\w+).py", module_name)
        return file_name.group(1)
    
    
    def get_file_name(self)->str:
        """
        Retrieves the name of the module associated with the logger.

        Returns:
            str: The name of the module associated with the logger.

        Example:
            ... in salat.py
            >>> logger = GlobalLogger()
            >>> logger.get_file_name()
            'salat'
        """
        return self.__file_name

    
    def message(self, msg:str)->None:
        """
        Logs a message associated with the calling module.

        Args:
            msg (str): The message to log.

        Returns:
            None

        Example:
            >>> logger = GlobalLogger()
            >>> logger.message("No onions!")
        """
        self.__messages_dict[self.get_file_name()].append(msg)

    
    def __getitem__(self, key:str)->str:
        """
        Retrieves the log messages associated with a specific module.

        Args:
            key (str): The name of the module.

        Returns:
            str: The log messages associated with the specified module.

        Example:
            ... in salat.py
            >>> logger = GlobalLogger()
            >>> logger["salat"]
            '[salat] - ['This in an info message.', 'This is an error message.', 'This message also shows as an error.', 'No onions!']'
        """
        return f"[{self.get_file_name()}] - {self.__messages_dict[key]}"
    
    
    def __get_log_level(self)->str:
        """
        Retrieves the log level of the logger.

        Returns:
            str: The log level of the logger.
        """
        return self.__log_level
    
    
    def log(self, message:str)->None:
        """
        Logs a message with the specified log level.

        Args:
            message (str): The message to log.

        Returns:
            None

        Example:
            >>> logger = GlobalLogger()
            >>> logger.log("This is an info message.")
        """
        print(f"[{self.__get_log_level()}] - {message}")
```

```python
if __name__ == "__main__":
    """
    Main block for testing the functionality of the GlobalLogger class.
    """
    logger1:GlobalLogger = GlobalLogger()

    logger1.log("This is an info message.")
    logger1.message("This in an info message.")
    print(logger1["salat"])

    logger2:GlobalLogger = GlobalLogger()

    logger2.log("This is an error message.")
    logger2.message("This is an error message.")
    logger1.log("This message also shows as an error.")
    logger1.message("This message also shows as an error.")
    print(logger2["salat"])

    myLogger:GlobalLogger = GlobalLogger()

    myLogger.message("No onions!")

    print(myLogger["salat"])

```

---
- output:
```
[Running] python -u "./salat.py"
[LOG] - This is an info message.
[salat] - ['This in an info message.']
[LOG] - This is an error message.
[LOG] - This message also shows as an error.
[salat] - ['This in an info message.', 'This is an error message.', 'This message also shows as an error.']
[salat] - ['This in an info message.', 'This is an error message.', 'This message also shows as an error.', 'No onions!']

[Done] exited with code=0 in 0.108 seconds
```

---
